# Mello ML - Fresh Unified Personality System

This notebook demonstrates the complete pipeline:
- **Unified personality approach**: Cultural data informs personality traits
- **768D embeddings**: Interests + 5 personality traits
- **50 archetypes**: Diverse synthetic user generation
- **Real user support**: Load from JSON files
- **2D visualization**: PCA and UMAP plotting

## 📦 Imports and Setup

In [1]:
%load_ext autoreload
%autoreload 2
# Interactive Plotly visualizations
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

# Core imports - Plotly for interactive visualization
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import logging
from IPython.display import display, HTML

# Fresh system components
from user import User
from profile_generator import ProfileGenerator
from embedding_generator import EmbeddingGenerator
from population import Population
from visualizer import Visualizer

# Configure logging
logging.basicConfig(level=logging.WARNING)  # Reduce noise

# Initialize components
print("🚀 Initializing Mello ML Components...")
profile_generator = ProfileGenerator()
embedding_generator = EmbeddingGenerator()
population = Population("Mello Campus Population")
visualizer = Visualizer()

print(f"✅ Components initialized:")
print(f"   📝 ProfileGenerator: {profile_generator.model}")
print(f"   🔢 EmbeddingGenerator: {embedding_generator}")
print(f"   📊 Architecture: 768D interests + 5×768D traits = 4608D combined")
print(f"   🎭 Archetypes available: {len(profile_generator.archetypes)}")
print(f"   📏 Similarity metric: Euclidean Distance (L2 norm, normalized, 0=different, 1=identical)")
print(f"   🎨 Visualization: Interactive Plotly interface")

/Users/yahyarahhawi/miniforge3/envs/ml/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🚀 Initializing Mello ML Components...
✅ Components initialized:
   📝 ProfileGenerator: google/gemini-2.5-flash
   🔢 EmbeddingGenerator: EmbeddingGenerator(model=text-embedding-004, dims=768, requests=0)
   📊 Architecture: 768D interests + 5×768D traits = 4608D combined
   🎭 Archetypes available: 50
   📏 Similarity metric: Euclidean Distance (L2 norm, normalized, 0=different, 1=identical)
   🎨 Visualization: Interactive Plotly interface


## 🎭 Generate 50 Synthetic Users

Creates diverse synthetic users using 50 personality archetypes with unified personality profiling.

In [2]:
print("🎭 Generating 50 synthetic users with unified personality approach...")
print("This may take several minutes due to API calls...")
print()

synthetic_users_generated = 0
target_count = 5

for i in range(target_count):
    print(f"   Generating user {i+1}/{target_count}...", end=" ")
    
    try:
        # Generate synthetic user data
        user_data = profile_generator.generate_synthetic_user_data()
        
        if user_data:
            # Create user from generated data
            user = User.from_json_data(user_data)
            
            # Generate unified profiles (interests + personality)
            profiles_success = profile_generator.generate_complete_profiles(user)
            
            if profiles_success:
                # Generate embeddings (768D each)
                embeddings_success = embedding_generator.embed_user_complete(user)
                
                if embeddings_success:
                    population.add_user(user)
                    synthetic_users_generated += 1
                    archetype = user_data.get('metadata', {}).get('archetype', 'Unknown')
                    print(f"✅ {user.name} ({archetype[:30]}...)")
                else:
                    print(f"❌ Failed embeddings")
            else:
                print(f"❌ Failed profiles")
        else:
            print(f"❌ Failed data generation")
    
    except Exception as e:
        print(f"❌ Error: {e}")
    
    # Progress update every 10 users
    if (i + 1) % 10 == 0:
        print(f"\n📈 Progress: {synthetic_users_generated}/{i + 1} users completed\n")

print(f"\n🎉 Synthetic user generation complete!")
print(f"✅ Successfully generated: {synthetic_users_generated}/{target_count} users")
print(f"📊 Success rate: {synthetic_users_generated/target_count*100:.1f}%")
print(f"👥 Population size: {len(population)} users")

🎭 Generating 50 synthetic users with unified personality approach...
This may take several minutes due to API calls...

   Generating user 1/5... 

KeyboardInterrupt: 

## 👤 Load Real User

Loads a real user from JSON file and processes through the same pipeline.

In [3]:
# Load real user from JSON file
real_user_path = "yahya_profile.json"

print(f"👤 Loading real user from: {real_user_path}")

try:
    # Load user from JSON
    real_user = User.from_json_file(real_user_path)
    real_user.special = True  # Mark as special for visualization
    
    print(f"✅ Loaded user: {real_user.name}")
    print(f"   Major: {real_user.profile_data.get('major', 'Unknown')}")
    print(f"   Bio: {real_user.profile_data.get('bio', 'No bio')[:100]}...")
    print(f"   Interests: {', '.join(real_user.profile_data.get('interests', [])[:5])}")
    
    # Generate unified personality profile from cultural data
    print(f"\n🔄 Processing {real_user.name} through unified pipeline...")
    print(f"   1. Generating unified interests profile from cultural preferences...")
    
    profiles_success = profile_generator.generate_complete_profiles(real_user)
    
    if profiles_success:
        print(f"   ✅ Generated unified profiles")
        
        # Show profile preview
        if real_user.interests_profile:
            print(f"   📖 Interests profile: {real_user.interests_profile[:150]}...")
        
        if real_user.personality_profiles:
            print(f"   🧠 Personality traits: {list(real_user.personality_profiles.keys())}")
        
        # Generate embeddings
        print(f"   2. Generating 768D embeddings...")
        embeddings_success = embedding_generator.embed_user_complete(real_user)
        
        if embeddings_success:
            print(f"   ✅ Generated embeddings (6 × 768D)")
            
            # Verify embedding dimensions
            combined = real_user.get_combined_embedding()
            if combined is not None:
                print(f"   🔢 Combined embedding shape: {combined.shape}")
            
            # Add to population
            population.add_user(real_user)
            print(f"   ✅ Added to population")
            
        else:
            print(f"   ❌ Failed to generate embeddings")
    else:
        print(f"   ❌ Failed to generate profiles")

except FileNotFoundError:
    print(f"❌ File not found: {real_user_path}")
    print(f"   Please ensure the JSON file exists in the correct location")
except Exception as e:
    print(f"❌ Error loading real user: {e}")

print(f"\n👥 Final population: {len(population)} users")
print(f"📊 Users with embeddings: {len(population.get_users_with_embeddings())}")

👤 Loading real user from: yahya_profile.json
✅ Loaded user: Yahya Rahhawi
   Major: Computer science, Philosophy
   Bio: I like philosophy, A.I, volleyball, soccer, watching movies and TV shows, and photography...
   Interests: philosophy, sports, computer science, machine learning, reading

🔄 Processing Yahya Rahhawi through unified pipeline...
   1. Generating unified interests profile from cultural preferences...
   ✅ Generated unified profiles
   📖 Interests profile: This individual possesses a profoundly introspective and analytical mind, constantly grappling with the fundamental questions of existence, morality, ...
   🧠 Personality traits: ['Openness', 'Conscientiousness', 'Extraversion', 'Agreeableness', 'Neuroticism']
   2. Generating 768D embeddings...
   ✅ Generated embeddings (6 × 768D)
   🔢 Combined embedding shape: (4608,)
   ✅ Added to population

👥 Final population: 6 users
📊 Users with embeddings: 6


In [3]:
population = Population()
population = population.load_from_json("mello_population.json")

## 📊 Population Statistics

Analyze the generated population and embedding quality.

In [4]:
# Get population statistics
stats = population.get_statistics()

print("📊 Population Statistics")
print("=" * 30)
#print(f"Population Name: {stats['population_name']}")
print(f"Total Users: {stats['total_users']}")
print(f"Users with Profiles: {stats['users_with_profiles']}")
print(f"Users with Embeddings: {stats['users_with_embeddings']}")

if stats['embedding_stats']:
    print(f"\n🔢 Embedding Dimensions:")
    for key, value in stats['embedding_stats'].items():
        if isinstance(value, int):
            print(f"   {key}: {value}D")
        elif isinstance(value, dict):
            print(f"   {key}:")
            for trait, dims in value.items():
                print(f"     {trait}: {dims}D")

# Find special users
special_users = [user for user in population.users if user.special]
print(f"\n⭐ Special Users: {len(special_users)}")
for user in special_users:
    print(f"   {user.name} - {user.profile_data.get('major', 'Unknown major')}")

# Embedding summary
embedding_summary = visualizer.create_embedding_summary(population)
print(f"\n🎯 Embedding Modes Available:")
for mode, info in embedding_summary['embedding_modes'].items():
    if 'users_count' in info and info['users_count'] > 0:
        print(f"   {mode}: {info['users_count']} users, {info.get('dimensions', '?')}D")

📊 Population Statistics
Total Users: 6
Users with Profiles: 6
Users with Embeddings: 6

🔢 Embedding Dimensions:
   interests_dims: 768D
   trait_dims:
     Openness: 768D
     Conscientiousness: 768D
     Extraversion: 768D
     Agreeableness: 768D
     Neuroticism: 768D
   combined_dims: 4608D

⭐ Special Users: 1
   Yahya Rahhawi - Computer science, Philosophy

🎯 Embedding Modes Available:
   combined: 6 users, 4608D
   interests: 6 users, 768D
   Openness: 6 users, 768D
   Conscientiousness: 6 users, 768D
   Extraversion: 6 users, 768D
   Agreeableness: 6 users, 768D
   Neuroticism: 6 users, 768D


## 🔍 Similarity Analysis

Test similarity search with the real user (if loaded).

In [5]:
# Find the real user for similarity testing
real_user = None
for user in population.users:
    if user.special:
        real_user = user
        break

if real_user and len(population.get_users_with_embeddings()) >= 5:
    print(f"🔍 Similarity Analysis for {real_user.name}")
    print("=" * 50)
    
    # Combined similarity (all embeddings)
    print(f"\n🎯 Most Similar Users (Combined Embeddings):")
    similar_combined = population.find_similar_users(real_user, mode='combined', top_k=5)
    
    for i, (similar_user, score) in enumerate(similar_combined, 1):
        archetype = similar_user.metadata.get('original_data', {}).get('metadata', {}).get('archetype', 'Unknown')
        print(f"   {i}. {similar_user.name}: {score:.3f} ({archetype[:40]}...)")
    
    # Interests similarity
    print(f"\n📚 Most Similar Users (Interests Only):")
    similar_interests = population.find_similar_users(real_user, mode='interests', top_k=5)
    
    for i, (similar_user, score) in enumerate(similar_interests, 1):
        archetype = similar_user.metadata.get('original_data', {}).get('metadata', {}).get('archetype', 'Unknown')
        print(f"   {i}. {similar_user.name}: {score:.3f} ({archetype[:40]}...)")
    
    # Trait-specific similarities
    print(f"\n🧠 Trait-Specific Most Similar Users:")
    traits = ['Openness', 'Conscientiousness', 'Extraversion', 'Agreeableness', 'Neuroticism']
    
    for trait in traits:
        try:
            similar_trait = population.find_similar_users(real_user, mode=trait, top_k=1)
            if similar_trait:
                most_similar, score = similar_trait[0]
                print(f"   {trait}: {most_similar.name} ({score:.3f})")
        except Exception as e:
            print(f"   {trait}: Error - {str(e)[:50]}...")

else:
    print(f"⚠️  Cannot perform similarity analysis:")
    if not real_user:
        print(f"   - No real user loaded (special=True)")
    if len(population.get_users_with_embeddings()) < 5:
        print(f"   - Need at least 5 users with embeddings (have {len(population.get_users_with_embeddings())})")

🔍 Similarity Analysis for Yahya Rahhawi

🎯 Most Similar Users (Combined Embeddings):
   1. Willie Reid: 0.994 (timeless classicist who values enduring ...)
   2. John Freeman: 0.994 (nostalgic traditionalist preferring time...)
   3. Amy Small: 0.994 (social activist interested in diverse vo...)
   4. Kimberly Jackson: 0.994 (methodical planner who enjoys structured...)
   5. Amy Mcdonald: 0.994 (fantasy escapist who prefers magical wor...)

📚 Most Similar Users (Interests Only):
   1. Willie Reid: 0.987 (timeless classicist who values enduring ...)
   2. Amy Small: 0.986 (social activist interested in diverse vo...)
   3. John Freeman: 0.985 (nostalgic traditionalist preferring time...)
   4. Kimberly Jackson: 0.984 (methodical planner who enjoys structured...)
   5. Amy Mcdonald: 0.983 (fantasy escapist who prefers magical wor...)

🧠 Trait-Specific Most Similar Users:
   Openness: John Freeman (0.989)
   Conscientiousness: Amy Small (0.986)
   Extraversion: Willie Reid (0.988)
   Agr

In [6]:
# Interactive Plotly Visualizations
users_with_embeddings = population.get_users_with_embeddings()

print(f"📊 Interactive Plotly Population Visualization")
print(f"Users with embeddings: {len(users_with_embeddings)}")
print(f"Similarity metric: Euclidean Distance (L2 norm, normalized)")
print()

if len(users_with_embeddings) >= 3:
    
    # 📚 Plotly PCA Visualization - Interests Only
    print(f"📚 Creating Plotly PCA - Interests Embeddings (768D → 2D):")
    try:
        fig_interests = visualizer.plot_population_pca(
            population, 
            mode='interests', 
            highlight_special=True, 
            figsize=(12, 8)
        )
        fig_interests.show()
        print(f"✅ Plotly interests PCA complete")
        
    except Exception as e:
        print(f"❌ Interests PCA failed: {e}")
    
    print()
    
    # 🧠 Plotly PCA Visualization - Combined Personality Traits
    print(f"🧠 Creating Plotly PCA - Combined Personality Traits (3840D → 2D):")
    try:
        # Get combined personality embedding (all 5 traits)
        from sklearn.decomposition import PCA
        from sklearn.preprocessing import StandardScaler
        import plotly.graph_objects as go
        import numpy as np
        
        # Get users with complete personality embeddings
        personality_users = []
        personality_embeddings = []
        
        for user in users_with_embeddings:
            trait_embeddings = []
            has_all_traits = True
            
            for trait in ['Openness', 'Conscientiousness', 'Extraversion', 'Agreeableness', 'Neuroticism']:
                trait_emb = getattr(user, f'{trait.lower()}_embedding', None)
                if trait_emb is not None:
                    trait_embeddings.append(trait_emb)
                else:
                    has_all_traits = False
                    break
            
            if has_all_traits:
                personality_users.append(user)
                combined_personality = np.concatenate(trait_embeddings)
                personality_embeddings.append(combined_personality)
        
        if len(personality_embeddings) >= 3:
            personality_matrix = np.array(personality_embeddings)
            
            # Apply PCA to personality embeddings
            scaler = StandardScaler()
            embeddings_scaled = scaler.fit_transform(personality_matrix)
            pca = PCA(n_components=2)
            embeddings_2d = pca.fit_transform(embeddings_scaled)
            
            # Create Plotly figure for personality
            fig_personality = go.Figure()
            
            # Separate special and regular users
            special_indices = []
            regular_indices = []
            
            for i, user in enumerate(personality_users):
                if user.special:
                    special_indices.append(i)
                else:
                    regular_indices.append(i)
            
            # Plot regular users
            if regular_indices:
                regular_coords = embeddings_2d[regular_indices]
                regular_names = [personality_users[i].name for i in regular_indices]
                
                fig_personality.add_trace(go.Scatter(
                    x=regular_coords[:, 0],
                    y=regular_coords[:, 1],
                    mode='markers',
                    marker=dict(size=8, color='lightgreen', opacity=0.7, line=dict(width=1, color='darkgreen')),
                    name=f'Users ({len(regular_indices)})',
                    hovertext=regular_names,
                    hovertemplate='<b>%{hovertext}</b><extra></extra>'
                ))
            
            # Plot special users
            if special_indices:
                special_coords = embeddings_2d[special_indices]
                special_names = [personality_users[i].name for i in special_indices]
                
                fig_personality.add_trace(go.Scatter(
                    x=special_coords[:, 0],
                    y=special_coords[:, 1],
                    mode='markers',
                    marker=dict(size=15, color='red', opacity=0.9, symbol='star', line=dict(width=2, color='darkred')),
                    name=f'Special Users ({len(special_indices)})',
                    hovertext=special_names,
                    hovertemplate='<b>%{hovertext}</b><extra></extra>'
                ))
            
            # Update layout
            total_variance = pca.explained_variance_ratio_[:2].sum()
            fig_personality.update_layout(
                title=f'PCA Visualization - Personality Traits Embeddings<br>{population.name} ({len(personality_users)} users)',
                xaxis_title=f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)',
                yaxis_title=f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)',
                hovermode='closest',
                showlegend=True,
                width=960,
                height=640,
                annotations=[
                    dict(
                        text=f'Total variance explained: {total_variance:.1%}<br>Similarity metric: Euclidean Distance<br>Dimensions: 5 traits × 768D = 3840D',
                        xref="paper", yref="paper",
                        x=0.02, y=0.98, xanchor='left', yanchor='top',
                        showarrow=False,
                        font=dict(size=12),
                        bgcolor="rgba(255,255,255,0.8)",
                        bordercolor="rgba(0,0,0,0.5)",
                        borderwidth=1
                    )
                ]
            )
            
            fig_personality.show()
            print(f"✅ Plotly personality traits PCA complete")
        else:
            print(f"❌ Need at least 3 users with complete personality embeddings, got {len(personality_embeddings)}")
        
    except Exception as e:
        print(f"❌ Personality PCA failed: {e}")

else:
    print(f"⚠️  Need at least 3 users with embeddings for visualization")
    print(f"   Current: {len(users_with_embeddings)} users")
    print(f"   Generate more synthetic users in the previous cell")

print(f"\n📖 Visualization Guide:")
print(f"   📚 Interests PCA: Cultural preferences embeddings (books, movies, music)")
print(f"   🧠 Personality PCA: Big 5 personality traits embeddings (Openness, Conscientiousness, etc.)")
print(f"   ⭐ Red stars: Real users (special)")
print(f"   🔵 Blue/Green dots: Synthetic users")
print(f"   📏 Similarity: Euclidean distance (1.0 = identical, 0.0 = different)")
print(f"   🎨 Interactive: Hover for names, zoom, pan to explore")

📊 Interactive Plotly Population Visualization
Users with embeddings: 6
Similarity metric: Euclidean Distance (L2 norm, normalized)

📚 Creating Plotly PCA - Interests Embeddings (768D → 2D):


✅ Plotly interests PCA complete

🧠 Creating Plotly PCA - Combined Personality Traits (3840D → 2D):


✅ Plotly personality traits PCA complete

📖 Visualization Guide:
   📚 Interests PCA: Cultural preferences embeddings (books, movies, music)
   🧠 Personality PCA: Big 5 personality traits embeddings (Openness, Conscientiousness, etc.)
   ⭐ Red stars: Real users (special)
   🔵 Blue/Green dots: Synthetic users
   📏 Similarity: Euclidean distance (1.0 = identical, 0.0 = different)
   🎨 Interactive: Hover for names, zoom, pan to explore


## 💾 Save Population

Save the complete population for future use.

In [7]:
# Save population to JSON
save_path = "mello_population.json"

print(f"💾 Saving population to {save_path}...")

try:
    population.save_to_json(save_path)
    print(f"✅ Population saved successfully")
    
    # Show file info
    import os
    file_size = os.path.getsize(save_path)
    print(f"   File size: {file_size:,} bytes ({file_size/1024/1024:.1f} MB)")
    print(f"   Users saved: {len(population)}")
    print(f"   Users with embeddings: {len(population.get_users_with_embeddings())}")
    
except Exception as e:
    print(f"❌ Failed to save population: {e}")

print(f"\n🎉 Notebook complete!")
print(f"📊 Final Statistics:")
print(f"   Population: {len(population)} users")
print(f"   Architecture: 768D interests + 5×768D traits")
print(f"   Approach: Unified personality profiling")
print(f"   Embeddings: {len(population.get_users_with_embeddings())} users ready")

💾 Saving population to mello_population.json...
✅ Population saved successfully
   File size: 738,622 bytes (0.7 MB)
   Users saved: 6
   Users with embeddings: 6

🎉 Notebook complete!
📊 Final Statistics:
   Population: 6 users
   Architecture: 768D interests + 5×768D traits
   Approach: Unified personality profiling
   Embeddings: 6 users ready
